In [1]:
import Pkg; Pkg.add("Revise"); using Revise; Pkg.activate("."); Pkg.instantiate();

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `/cache/polymake/oscar-system/Oscar.jl/Project.toml`
  No Changes to `/cache/polymake/oscar-system/Oscar.jl/Manifest.toml`
Precompiling project...
  ✓ Oscar
  1 dependency successfully precompiled in 23 seconds (66 already precompiled)
  Activating project at `/cache/polymake/oscar-system/Oscar.jl`


In [2]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.8.2-DEV ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


In [3]:
C = cube(3)

LC = ehrhart_polynomial(C)
factor(LC)

1 * (2*x + 1)^3

In [4]:
galois_group(LC)

(Group(()), Galois Context for 2*x + 1 and prime 3)

In [5]:
S = fano_simplex(14)

A polyhedron in ambient dimension 14

In [6]:
@time G = galois_group(ehrhart_polynomial(S))

  4.803177 seconds (12.17 M allocations: 733.616 MiB, 3.82% gc time, 73.94% compilation time)


(<permutation group of size 645120 with 9 generators>, Galois Context for 15*x^14 + 105*x^13 + 24115*x^12 + 143325*x^11 + 7724717*x^10 + 37312275*x^9 + 725938785*x^8 + 2681453775*x^7 + 21964438496*x^6 + 56663366760*x^5 + 201186840400*x^4 + 310989260400*x^3 + 429952217472*x^2 + 283465647360*x + 87178291200 and prime 59)

In [7]:
Qt, t = QQ["t"]

(Univariate Polynomial Ring in t over Rational Field, t)

In [8]:
LS(d) = binomial(t+d+1,d+1) - binomial(t,d+1)

LS (generic function with 1 method)

In [9]:
factorial(6) * LS(6)

7*t^6 + 21*t^5 + 385*t^4 + 735*t^3 + 2128*t^2 + 1764*t + 720

In [10]:
roots(LS(6), ComplexField(64))

6-element Vector{acb}:
 [-0.50000000000000000000 +/- 1e-25] + [-0.4466484333276384126 +/- 7.42e-20]*im
 [-0.50000000000000000000 +/- 1e-25] + [0.4466484333276384126 +/- 7.42e-20]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [-2.157723183499887922 +/- 3.77e-19]*im
 [-0.50000000000000000000 +/- 1e-25] + [2.157723183499887922 +/- 3.77e-19]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [-6.811368132790436334 +/- 7.10e-19]*im
 [-0.5000000000000000000 +/- 5.43e-20] + [6.811368132790436334 +/- 7.10e-19]*im

In [11]:
describe(galois_group(LS(6))[1])

"C2 x S4"

In [12]:
db = Polymake.Polydb.get_db();
collection = db["Polytopes.Lattice.SmoothReflexive"];
query = Dict("DIM"=>6);
results = Polymake.Polydb.find(collection, query);

In [13]:
PP6 = [ polarize(Polyhedron(P)) for P in results ]
E6 = [ ehrhart_polynomial(P) for P in PP6 ];

In [14]:
size(E6)

(7622,)

In [15]:
sort(MSet(order(galois_group(e)[1]) for e = E6).dict)

OrderedCollections.OrderedDict{Any, Int64} with 6 entries:
  4  => 623
  6  => 1
  8  => 22
  12 => 44
  16 => 310
  48 => 6622

In [16]:
function on_critical_line(L)    
    for z in roots(L, ComplexField(64))
        if Float64(real(z)) != -1/2
            return false
        end
    end
    return true
end

on_critical_line (generic function with 1 method)

In [17]:
on_critical_line(LS(6))

true

In [18]:
function check_critical_line(EE)
    idx = 1
    non_critical = []
    for e in EE
        if !on_critical_line(e)
            push!(non_critical, idx)
        end
        idx+=1
    end
    return non_critical
end

check_critical_line (generic function with 1 method)

In [19]:
NC6 = check_critical_line(E6)

4-element Vector{Any}:
  721
  804
 2617
 3155

In [20]:
[ describe(galois_group(E6[k])[1]) for k in NC6 ]

4-element Vector{String}:
 "C2 x S4"
 "C2 x S4"
 "C2 x S4"
 "S3"